In [ ]:
!pip install pyGMM

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.5/954.5 kB 8.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [ ]:
# Data
data = pd.read_excel('Cyprus_nM=50,dtp=15.xlsx')

In [ ]:
# Shuffle Data
shuffled_data = data.sample(frac=1, random_state=42)

# Reset the index to avoid any issues with the shuffled data
shuffled_data.reset_index(drop=True, inplace=True)

data = shuffled_data

In [ ]:
# Change response to categorical
column_to_bin = 'Actual Mag'
bin_edges = [0, 3.0, 5.0, 6.0, 10.0]

bin_labels = ['Small', 'Medium', 'Large', 'Catastrophic']

data['Binned_Column'] = pd.cut(data[column_to_bin], bins=bin_edges, labels=bin_labels)

In [ ]:
# Train Model
net = MLPClassifier(hidden_layer_sizes=(20,), activation='logistic', max_iter=400)

In [ ]:
# Split your data into features (X) and labels (y)
X = data.iloc[:, :60].values
y = data.iloc[:, 60].values

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=63)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.6, random_state=63)

# Standardize the input data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Create a neural network model
model = Sequential()
model.add(Dense(64, input_dim=60, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['MSE'])

# Train the model
run = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=200, batch_size=32)


Epoch 1/200
138/138 [==============================] - 3s 7ms/step - loss: -100.1207 - MSE: 6.7579 - val_loss: -412.9475 - val_MSE: 6.6258
Epoch 2/200
138/138 [==============================] - 1s 4ms/step - loss: -1895.6426 - MSE: 6.5129 - val_loss: -4160.7690 - val_MSE: 6.6258
Epoch 3/200
138/138 [==============================] - 0s 2ms/step - loss: -9152.4131 - MSE: 6.5129 - val_loss: -15059.6172 - val_MSE: 6.6258
Epoch 4/200
138/138 [==============================] - 0s 3ms/step - loss: -25563.3906 - MSE: 6.5129 - val_loss: -36372.5312 - val_MSE: 6.6258
Epoch 5/200
138/138 [==============================] - 0s 2ms/step - loss: -54032.8242 - MSE: 6.5129 - val_loss: -70659.2188 - val_MSE: 6.6258
Epoch 6/200
138/138 [==============================] - 1s 6ms/step - loss: -97654.9062 - MSE: 6.5129 - val_loss: -120979.5781 - val_MSE: 6.6258
Epoch 7/200
138/138 [==============================] - 1s 8ms/step - loss: -158917.7969 - MSE: 6.5129 - val_loss: -189061.2344 - val_MSE: 6.6258
Epo

In [ ]:
# Get Paramter Values for GMM function
import tensorflow as tf

params = model.get_weights()

"""
print(params[4][23][0]) #vs_30
print(params[3][30]) # dist_rup
print(params[2][1][31]) #mag
"""

vs30 = params[4][23][0]
dist_rup = params[3][30]
mag = params[2][1][31]

In [ ]:
"""Idriss (2014, :cite:`idriss14`) model."""
import numpy as np

from pygmm import model

__author__ = "Albert Kottke"


class Idriss2014(model.GroundMotionModel):
    """Idriss (2014, :cite:`idriss14`) model.

    This model was developed for active tectonic regions as part of the
    NGA-West2 effort.

    Parameters
    ----------
    scenario : :class:`pygmm.model.Scenario`
        earthquake scenario

    """

    NAME = "Idriss (2014)"
    ABBREV = "I14"

    # Reference velocity (m/s)
    V_REF = 1200.0

    # Load the coefficients for the model
    COEFF = dict(
        small=model.load_data_file("idriss_2014-small.csv", 2),
        large=model.load_data_file("idriss_2014-large.csv", 2),
    )
    PERIODS = COEFF["small"]["period"]

    INDEX_PGA = 0
    INDICES_PSA = np.arange(22)

    PARAMS = [
        model.NumericParameter("dist_rup", True, None, 150),
        model.NumericParameter("mag", True, 5, None),
        model.NumericParameter("v_s30", True, 450, 1200),
        model.CategoricalParameter("mechanism", True, ["SS", "RS"], "SS"),
    ]


    def __init__(self, scenario: model.Scenario):
        """Initialize the model."""
        super().__init__(scenario)
        self._ln_resp = self._calc_ln_resp()
        self._ln_std = self._calc_ln_std()



    def _calc_ln_resp(self) -> np.ndarray:
        """Calculate the natural logarithm of the response.

        Returns
        -------
        ln_resp : class:`np.array`:
            natural log of the response

        """
        s = self._scenario
        c = self.COEFF["small"] if s.mag <= 6.75 else self.COEFF["large"]

        if s.mechanism == "RS":
            flag_mech = 1
        else:
            # SS/RS/U
            flag_mech = 0

        f_mag = c.alpha_1 + c.alpha_2 * s.mag + c.alpha_3 * (8.5 - s.mag) ** 2
        f_dst = (
            -(c.beta_1 + c.beta_2 * s.mag) * np.log(s.dist_rup + 10)
            + c.gamma * s.dist_rup
        )
        f_ste = c.epsilon * np.log(s.v_s30)
        f_mec = c.phi * flag_mech

        ln_resp = f_mag + f_dst + f_ste + f_mec

        return ln_resp

    def _calc_ln_std(self) -> np.ndarray:
        """Calculate the logarithmic standard deviation.

        Returns
        -------
        ln_std : class:`np.array`:
            natural log standard deviation

        """
        s = self._scenario
        ln_std = (
            1.18
            + 0.035 * np.log(np.clip(self.PERIODS, 0.05, 3.0))
            - 0.06 * np.clip(s.mag, 5.0, 7.5)
        )
        return ln_std

In [ ]:
from pygmm.model import Scenario

In [ ]:
# Create an instance of the predictors
scenario = Scenario(
    mag= mag,
    dist_rup= dist_rup,
    v_s30=vs30,
    mechanism="SS"
)

In [ ]:
model = Idriss2014(scenario)

<ipython-input-99-594425a62087>:77: RuntimeWarning: invalid value encountered in log
  f_ste = c.epsilon * np.log(s.v_s30)


In [ ]:
# Calculate Metrics
ln_resp = model._calc_ln_resp()
ln_std = model._calc_ln_std()
# Print Accuracy
print(np.mean(ln_std))

0.6961553421379612


<ipython-input-99-594425a62087>:77: RuntimeWarning: invalid value encountered in log
  f_ste = c.epsilon * np.log(s.v_s30)
